In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import os
import shutil

# Define paths
base_dir = 'D:/PKG - C-NMC 2019/C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = 'D:/PKG - C-NMC 2019/C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 64, 64  # Reduced image size
batch_size = 16  # Increased batch size
epochs = 10  # Reduced number of epochs

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            for filename in os.listdir(category_dir):
                if filename.endswith('.bmp'):
                    src = os.path.join(category_dir, filename)
                    dst = os.path.join(combined_category_dir, filename)
                    if not os.path.exists(dst):  # Avoid duplicates
                        shutil.copy(src, dst)

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generators
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation (test)

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for testing
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# Define a smaller CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train the model with the combined dataset
print("Training with combined dataset")
model = create_model()
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs
)

# Save the model
model.save('model_combined.h5')
print("Model saved as model_combined.h5")

# Evaluate the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/10
633/633 [==============================] - 202s 317ms/step - loss: 0.5376 - accuracy: 0.7469 - val_loss: 0.3800 - val_accuracy: 0.8617
Epoch 2/10
633/633 [==============================] - 200s 316ms/step - loss: 0.4785 - accuracy: 0.7922 - val_loss: 0.3902 - val_accuracy: 0.8371
Epoch 3/10
633/633 [==============================] - 173s 274ms/step - loss: 0.4696 - accuracy: 0.7952 - val_loss: 0.3828 - val_accuracy: 0.8447
Epoch 4/10
633/633 [==============================] - 163s 257ms/step - loss: 0.4670 - accuracy: 0.8001 - val_loss: 0.3796 - val_accuracy: 0.8409
Epoch 5/10
633/633 [==============================] - 167s 264ms/step - loss: 0.4643 - accuracy: 0.8000 - val_loss: 0.3956 - val_accuracy: 0.8333
Epoch 6/10
633/633 [==============================] - 158s 250ms/step - loss: 0.4600 - accuracy: 0.8027 - val_loss: 0.3705 - val_accuracy: 0.8504
Epoch 7/1

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os
import shutil

# Define paths
base_dir = 'D:/PKG - C-NMC 2019/C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = 'D:/PKG - C-NMC 2019/C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 128, 128  # Reduced image size
batch_size = 32  # Increased batch size
epochs = 10  # Reduced number of epochs

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            for filename in os.listdir(category_dir):
                if filename.endswith('.bmp'):
                    src = os.path.join(category_dir, filename)
                    dst = os.path.join(combined_category_dir, filename)
                    if not os.path.exists(dst):  # Avoid duplicates
                        shutil.copy(src, dst)

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generators
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation (test)

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for testing
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# Define a smaller CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train the model with the combined dataset
print("Training with combined dataset")
model1 = create_model()
history = model1.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs
)

# Save the model
model1.save('model_improve_1_combined.h5')
print("Model saved as model_improve_1_combined.h5")

# Evaluate the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model1.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/10
316/316 [==============================] - 127s 398ms/step - loss: 0.5246 - accuracy: 0.7541 - val_loss: 0.4213 - val_accuracy: 0.8242
Epoch 2/10
316/316 [==============================] - 123s 388ms/step - loss: 0.4698 - accuracy: 0.7968 - val_loss: 0.4236 - val_accuracy: 0.8203
Epoch 3/10
316/316 [==============================] - 125s 395ms/step - loss: 0.4656 - accuracy: 0.7963 - val_loss: 0.4026 - val_accuracy: 0.8301
Epoch 4/10
316/316 [==============================] - 127s 401ms/step - loss: 0.4635 - accuracy: 0.7981 - val_loss: 0.4527 - val_accuracy: 0.8027
Epoch 5/10
316/316 [==============================] - 123s 389ms/step - loss: 0.4615 - accuracy: 0.8017 - val_loss: 0.4254 - val_accuracy: 0.8262
Epoch 6/10
316/316 [==============================] - 121s 383ms/step - loss: 0.4588 - accuracy: 0.8026 - val_loss: 0.4081 - val_accuracy: 0.8262
Epoch 7/1

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os
import shutil

# Define paths
base_dir = 'D:/PKG - C-NMC 2019/C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = 'D:/PKG - C-NMC 2019/C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 128, 128  # Image size
batch_size = 16  # Batch size
epochs = 20  # Number of epochs

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            for filename in os.listdir(category_dir):
                if filename.endswith('.bmp'):
                    src = os.path.join(category_dir, filename)
                    dst = os.path.join(combined_category_dir, filename)
                    if not os.path.exists(dst):  # Avoid duplicates
                        shutil.copy(src, dst)

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generators
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for validation
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# Define the CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for dynamic learning rate and early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-6,
    verbose=1
)

# Train the model with the combined dataset
print("Training with combined dataset")
model2 = create_model()
history = model2.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr_on_plateau]
)

# Save the model
model2.save('model_improve_2_combined.h5')
print("Model saved as model_improve_2_combined.h5")

# Evaluate the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model2.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/20
633/633 [==============================] - 171s 267ms/step - loss: 0.4899 - accuracy: 0.7834 - val_loss: 0.4298 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 2/20
633/633 [==============================] - 211s 334ms/step - loss: 0.4664 - accuracy: 0.7972 - val_loss: 0.4146 - val_accuracy: 0.8182 - lr: 0.0010
Epoch 3/20
633/633 [==============================] - 217s 343ms/step - loss: 0.4602 - accuracy: 0.8020 - val_loss: 0.3831 - val_accuracy: 0.8314 - lr: 0.0010
Epoch 4/20
633/633 [==============================] - 175s 276ms/step - loss: 0.4501 - accuracy: 0.8027 - val_loss: 0.4072 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 5/20
633/633 [==============================] - 160s 252ms/step - loss: 0.4519 - accuracy: 0.8042 - val_loss: 0.3871 - val_accuracy: 0.8466 - lr: 0.0010
Epoch 6/20
633/633 [==============================] - 164s 258ms/step - loss: 0.4465 - acc

In [21]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define paths
base_dir = r'D:\PKG - C-NMC 2019\C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = r'D:\PKG - C-NMC 2019\C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 128, 128  # Image size
batch_size = 16  # Batch size
epochs = 20  # Number of epochs

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            if os.path.exists(category_dir):
                for filename in os.listdir(category_dir):
                    if filename.endswith('.bmp'):
                        src = os.path.join(category_dir, filename)
                        dst = os.path.join(combined_category_dir, filename)
                        if not os.path.exists(dst):  # Avoid duplicates
                            shutil.copy(src, dst)
            else:
                print(f"Warning: The directory {category_dir} does not exist.")

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generators
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for validation
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# Define the CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for dynamic learning rate and early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-6,
    verbose=1
)

# Train the model with the combined dataset
print("Training with combined dataset")
model3 = create_model()
history = model3.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr_on_plateau]
)

# Save the model
model3.save('model_improve_3_combined.h5')
print("Model saved as model_improve_3_combined.h5")

# Evaluate the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model3.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/20
633/633 [==============================] - 179s 280ms/step - loss: 0.4921 - accuracy: 0.7835 - val_loss: 0.4101 - val_accuracy: 0.8239 - lr: 0.0010
Epoch 2/20
633/633 [==============================] - 160s 253ms/step - loss: 0.4652 - accuracy: 0.8001 - val_loss: 0.4162 - val_accuracy: 0.8182 - lr: 0.0010
Epoch 3/20
633/633 [==============================] - 149s 235ms/step - loss: 0.4576 - accuracy: 0.8026 - val_loss: 0.3881 - val_accuracy: 0.8258 - lr: 0.0010
Epoch 4/20
633/633 [==============================] - 198s 313ms/step - loss: 0.4565 - accuracy: 0.8020 - val_loss: 0.3777 - val_accuracy: 0.8295 - lr: 0.0010
Epoch 5/20
633/633 [==============================] - 157s 247ms/step - loss: 0.4465 - accuracy: 0.8086 - val_loss: 0.3678 - val_accuracy: 0.8314 - lr: 0.0010
Epoch 6/20
633/633 [==============================] - 158s 250ms/step - loss: 0.4433 - acc

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import os
import shutil

# Define paths
base_dir = 'D:/PKG - C-NMC 2019/C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = 'D:/PKG - C-NMC 2019/C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 128, 128  # Image size
batch_size = 16  # Batch size
epochs = 20  # Number of epochs

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            if os.path.exists(category_dir):
                for filename in os.listdir(category_dir):
                    if filename.endswith('.bmp'):
                        src = os.path.join(category_dir, filename)
                        dst = os.path.join(combined_category_dir, filename)
                        if not os.path.exists(dst):  # Avoid duplicates
                            shutil.copy(src, dst)

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generators
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for validation
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# Define the CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for dynamic learning rate and early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-6,
    verbose=1
)

# Train the model with the combined dataset
print("Training with combined dataset")
model4 = create_model()
history = model4.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr_on_plateau]
)

# Save the model
model4.save('model_improve_4_combined.h5')
print("Model saved as model_improve_4_combined.h5")

# Evaluate the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model4.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/20
633/633 [==============================] - 219s 342ms/step - loss: 0.4889 - accuracy: 0.7798 - val_loss: 0.4042 - val_accuracy: 0.8087 - lr: 0.0010
Epoch 2/20
633/633 [==============================] - 121s 190ms/step - loss: 0.4664 - accuracy: 0.8004 - val_loss: 0.3750 - val_accuracy: 0.8466 - lr: 0.0010
Epoch 3/20
633/633 [==============================] - 79s 125ms/step - loss: 0.4594 - accuracy: 0.8022 - val_loss: 0.4263 - val_accuracy: 0.8049 - lr: 0.0010
Epoch 4/20
633/633 [==============================] - 71s 112ms/step - loss: 0.4550 - accuracy: 0.8023 - val_loss: 0.4013 - val_accuracy: 0.8239 - lr: 0.0010
Epoch 5/20
633/633 [==============================] - ETA: 0s - loss: 0.4506 - accuracy: 0.8076
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
633/633 [==============================] - 71s 113ms/step - loss: 0.4506 - accu

In [27]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define paths
base_dir = r'D:\PKG - C-NMC 2019\C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = r'D:\PKG - C-NMC 2019\C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 128, 128  
batch_size = 16 
epochs = 30  

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            if os.path.exists(category_dir):
                for filename in os.listdir(category_dir):
                    if filename.endswith('.bmp'):
                        src = os.path.join(category_dir, filename)
                        dst = os.path.join(combined_category_dir, filename)
                        if not os.path.exists(dst):  # Avoid duplicates
                            shutil.copy(src, dst)
            else:
                print(f"Warning: The directory {category_dir} does not exist.")

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generator
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for validation
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3), activation='softmax', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Flatten(),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for dynamic learning rate and early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-6,
    verbose=1
)

# Train the model
print("Training with combined dataset")
model5 = create_model()
history = model5.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr_on_plateau]
)

# Save the model
model5.save('model_improve_5_combined.h5')
print("Model saved as model_improve_5_combined.h5")

# Evaluating the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model5.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/30
633/633 [==============================] - 151s 237ms/step - loss: 0.5187 - accuracy: 0.7625 - val_loss: 0.3856 - val_accuracy: 0.8390 - lr: 0.0010
Epoch 2/30
633/633 [==============================] - 145s 229ms/step - loss: 0.4730 - accuracy: 0.7921 - val_loss: 0.3848 - val_accuracy: 0.8598 - lr: 0.0010
Epoch 3/30
633/633 [==============================] - 164s 259ms/step - loss: 0.4672 - accuracy: 0.7952 - val_loss: 0.3760 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/30
633/633 [==============================] - 236s 373ms/step - loss: 0.4632 - accuracy: 0.7948 - val_loss: 0.4430 - val_accuracy: 0.8030 - lr: 0.0010
Epoch 5/30
633/633 [==============================] - 178s 281ms/step - loss: 0.4648 - accuracy: 0.7991 - val_loss: 0.4064 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 6/30
633/633 [==============================] - ETA: 0s - loss: 0.4609 - accuracy: 0

In [9]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define paths
base_dir = r'D:\PKG - C-NMC 2019\C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = r'D:\PKG - C-NMC 2019\C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 128, 128  
batch_size = 16 
epochs = 30  

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            if os.path.exists(category_dir):
                for filename in os.listdir(category_dir):
                    if filename.endswith('.bmp'):
                        src = os.path.join(category_dir, filename)
                        dst = os.path.join(combined_category_dir, filename)
                        if not os.path.exists(dst):  # Avoid duplicates
                            shutil.copy(src, dst)
            else:
                print(f"Warning: The directory {category_dir} does not exist.")

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generator
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for validation
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='softplus', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Flatten(),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for dynamic learning rate and early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-7,
    verbose=1
)

# Train the model
print("Training with combined dataset")
model6 = create_model()
history = model6.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr_on_plateau]
)

# Save the model
model6.save('model_improve_6_combined.h5')
print("Model saved as model_improve_6_combined.h5")

# Evaluating the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model6.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/30
633/633 [==============================] - 142s 222ms/step - loss: 0.5542 - accuracy: 0.7407 - val_loss: 0.4362 - val_accuracy: 0.8049 - lr: 0.0010
Epoch 2/30
633/633 [==============================] - 151s 239ms/step - loss: 0.5177 - accuracy: 0.7887 - val_loss: 0.3966 - val_accuracy: 0.8580 - lr: 0.0010
Epoch 3/30
633/633 [==============================] - 142s 225ms/step - loss: 0.4884 - accuracy: 0.7887 - val_loss: 0.3930 - val_accuracy: 0.8466 - lr: 0.0010
Epoch 4/30
633/633 [==============================] - 144s 227ms/step - loss: 0.4766 - accuracy: 0.7966 - val_loss: 0.4474 - val_accuracy: 0.8182 - lr: 0.0010
Epoch 5/30
633/633 [==============================] - 140s 221ms/step - loss: 0.4722 - accuracy: 0.7972 - val_loss: 0.4167 - val_accuracy: 0.8106 - lr: 0.0010
Epoch 6/30
633/633 [==============================] - ETA: 0s - loss: 0.4617 - accuracy: 0

In [10]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define paths
base_dir = r'D:\PKG - C-NMC 2019\C-NMC_training_data'
folds = ['fold_0', 'fold_1', 'fold_2']
combined_dir = r'D:\PKG - C-NMC 2019\C-NMC_combined_data'
all_dir = os.path.join(combined_dir, 'all')
hem_dir = os.path.join(combined_dir, 'hem')

# Parameters
img_width, img_height = 128, 128  
batch_size = 16 
epochs = 30  

# Function to combine directories
def combine_directories(base_dir, folds, all_dir, hem_dir):
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)
    if not os.path.exists(hem_dir):
        os.makedirs(hem_dir)
    
    for fold in folds:
        fold_dir = os.path.join(base_dir, fold)
        for category in ['all', 'hem']:
            category_dir = os.path.join(fold_dir, category)
            combined_category_dir = all_dir if category == 'all' else hem_dir
            if os.path.exists(category_dir):
                for filename in os.listdir(category_dir):
                    if filename.endswith('.bmp'):
                        src = os.path.join(category_dir, filename)
                        dst = os.path.join(combined_category_dir, filename)
                        if not os.path.exists(dst):  # Avoid duplicates
                            shutil.copy(src, dst)
            else:
                print(f"Warning: The directory {category_dir} does not exist.")

combine_directories(base_dir, folds, all_dir, hem_dir)

# Data Generator
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.05)  # 5% for validation

def create_data_generators(combined_dir, img_width, img_height, batch_size):
    train_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'  # 95% for training
    )
    test_generator = datagen.flow_from_directory(
        combined_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'  # 5% for validation
    )
    return train_generator, test_generator

train_generator, test_generator = create_data_generators(combined_dir, img_width, img_height, batch_size)

# CNN model
def create_model():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='softsign', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Flatten(),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Callbacks for dynamic learning rate and early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-7,
    verbose=1
)

# Train the model
print("Training with combined dataset")
model9 = create_model()
history = model9.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr_on_plateau]
)

# Save the model
model9.save('model_improve_9_combined.h5')
print("Model saved as model_improve_9_combined.h5")

# Evaluating the model on the test data
print("Evaluating the model on the test data")
test_loss, test_accuracy = model9.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Found 10129 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Training with combined dataset
Epoch 1/30
633/633 [==============================] - 149s 233ms/step - loss: 0.5012 - accuracy: 0.7788 - val_loss: 0.4063 - val_accuracy: 0.8466 - lr: 0.0010
Epoch 2/30
633/633 [==============================] - 139s 219ms/step - loss: 0.4713 - accuracy: 0.7980 - val_loss: 0.3931 - val_accuracy: 0.8314 - lr: 0.0010
Epoch 3/30
633/633 [==============================] - 154s 244ms/step - loss: 0.4627 - accuracy: 0.7987 - val_loss: 0.3631 - val_accuracy: 0.8466 - lr: 0.0010
Epoch 4/30
633/633 [==============================] - 167s 263ms/step - loss: 0.4609 - accuracy: 0.8025 - val_loss: 0.3909 - val_accuracy: 0.8220 - lr: 0.0010
Epoch 5/30
633/633 [==============================] - 155s 245ms/step - loss: 0.4537 - accuracy: 0.8021 - val_loss: 0.4205 - val_accuracy: 0.8182 - lr: 0.0010
Epoch 6/30
633/633 [==============================] - ETA: 0s - loss: 0.4523 - accuracy: 0

KeyboardInterrupt: 